In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
import matplotlib.pyplot as plt

if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    print('----------- Float tensor set --------------')
else:
    print('---------------------- No CUDA -----------------')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_name = "BAAI/bge-large-en-v1.5"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

/usr/local/lib/python3.10/dist-packages/torch/__init__.py:1144: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:432.)
  _C._set_default_tensor_type(t)


----------- Float tensor set --------------


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

In [38]:
print(model.encoder)

BertEncoder(
  (layer): ModuleList(
    (0-23): 24 x BertLayer(
      (attention): BertAttention(
        (self): BertSdpaSelfAttention(
          (query): Linear(in_features=1024, out_features=1024, bias=True)
          (key): Linear(in_features=1024, out_features=1024, bias=True)
          (value): Linear(in_features=1024, out_features=1024, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (output): BertSelfOutput(
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (intermediate): BertIntermediate(
        (dense): Linear(in_features=1024, out_features=4096, bias=True)
        (intermediate_act_fn): GELUActivation()
      )
      (output): BertOutput(
        (dense): Linear(in_features=4096, out_features=1024, bias=True)
        (LayerNorm): LayerNorm((1024,), eps=1e-12, elementw

In [13]:
targetText = 'magic is real'
print(tokenizer(targetText, return_tensors='pt'))
outputs = model(**tokenizer(targetText, return_tensors='pt'), output_hidden_states=True)
target_embedding = outputs.pooler_output[0]
print(target_embedding)

{'input_ids': tensor([[ 101, 3894, 2003, 2613,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}
tensor([-0.9450, -0.8097, -0.8362,  ...,  0.4859,  0.9817, -0.8716],
       grad_fn=<SelectBackward0>)


In [21]:
outputs.pooler_output[0].shape

torch.Size([1, 1024])

In [ ]:
for i in range(len(outputs.hidden_states)):
    print(outputs.hidden_states[i])

In [ ]:
outputs.pooler_output[0]

tensor([-0.9450, -0.8097, -0.8362,  ...,  0.4859,  0.9817, -0.8716],
       grad_fn=<SelectBackward0>)

In [ ]:
model.pooler(outputs.hidden_states[-1])[0]

tensor([-0.9450, -0.8097, -0.8362,  ...,  0.4859,  0.9817, -0.8716],
       grad_fn=<SelectBackward0>)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
import torch.nn.functional as F

def cosine_similarity(embedding1, embedding2):
  # Normalize the embeddings
  embedding1_normalized = F.normalize(embedding1, p=2, dim=0)
  embedding2_normalized = F.normalize(embedding2, p=2, dim=0)

  # Calculate the dot product
  dot_product = torch.dot(embedding1_normalized, embedding2_normalized)

  return dot_product.item()

text1 = 'magic is real'
text2 = 'magic are real'
print(tokenizer(text1, return_tensors='pt'))
print(tokenizer(text2, return_tensors='pt'))
cosine_similarity(model(**tokenizer(text1, return_tensors='pt'), output_hidden_states=True).pooler_output[0], model(**tokenizer(text2, return_tensors='pt'), output_hidden_states=True).pooler_output[0])

{'input_ids': tensor([[ 101, 3894, 2003, 2613,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}
{'input_ids': tensor([[ 101, 3894, 2024, 2613,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}


0.9960455894470215

In [ ]:
cosineSimilarities = []

for i in range(len(outputs.hidden_states)):
    hiddenStateSimilarity = []
    embeddingI = model.pooler(outputs.hidden_states[i])[0]
    for j in range(len(outputs.hidden_states)):
        embeddingJ = model.pooler(outputs.hidden_states[j])[0]
        hiddenStateSimilarity.append(round(cosine_similarity(embeddingI, embeddingJ), 2))
    cosineSimilarities.append(hiddenStateSimilarity)

In [ ]:
for i in range(len(cosineSimilarities)):
    print(f'{cosineSimilarities[i]}')

[1.0, 0.85, 0.84, 0.84, 0.85, 0.78, 0.59, 0.35, -0.0, -0.06, 0.01, 0.17, -0.01, 0.07, -0.08, 0.08, 0.1, -0.15, -0.17, -0.03, -0.01, 0.1, 0.11, 0.19, 0.16]
[0.85, 1.0, 1.0, 0.99, 0.99, 0.93, 0.7, 0.36, -0.05, -0.09, 0.05, 0.29, 0.07, 0.18, -0.01, 0.24, 0.26, -0.12, -0.2, -0.01, 0.01, 0.16, 0.17, 0.26, 0.27]
[0.84, 1.0, 1.0, 1.0, 0.99, 0.93, 0.71, 0.36, -0.07, -0.11, 0.04, 0.29, 0.06, 0.18, -0.01, 0.26, 0.29, -0.1, -0.19, 0.01, 0.04, 0.19, 0.2, 0.29, 0.3]
[0.84, 0.99, 1.0, 1.0, 1.0, 0.93, 0.7, 0.35, -0.08, -0.11, 0.03, 0.28, 0.05, 0.15, -0.03, 0.25, 0.28, -0.1, -0.18, 0.02, 0.04, 0.2, 0.21, 0.3, 0.3]
[0.85, 0.99, 0.99, 1.0, 1.0, 0.95, 0.74, 0.38, -0.09, -0.13, 0.02, 0.28, 0.05, 0.15, -0.04, 0.26, 0.29, -0.09, -0.18, 0.04, 0.06, 0.22, 0.23, 0.33, 0.33]
[0.78, 0.93, 0.93, 0.93, 0.95, 1.0, 0.91, 0.58, -0.04, -0.1, 0.08, 0.39, 0.12, 0.2, -0.02, 0.32, 0.37, -0.08, -0.19, 0.08, 0.1, 0.31, 0.31, 0.42, 0.45]
[0.59, 0.7, 0.71, 0.7, 0.74, 0.91, 1.0, 0.79, 0.07, -0.0, 0.18, 0.49, 0.21, 0.25, 0.01, 

In [36]:
import numpy as np

def find_all_possible_inputs(y, W, b):
    """
    Find all possible inputs x that satisfy y = tanh(Wx + b)

    Parameters:
    y: target output vector (must have all elements between -1 and 1)
    W: weight matrix
    b: bias vector

    Returns:
    function that generates solutions based on input z
    """
    # Check if y is valid (all elements between -1 and 1)
    if not np.all(np.abs(y) < 1):
        raise ValueError("All elements of y must be between -1 and 1")

    # Step 1: Apply arctanh
    arctanh_y = np.arctanh(y)

    # Step 2: Subtract bias
    target = arctanh_y - b

    # Step 3: Find pseudoinverse of W
    W_pinv = np.linalg.pinv(W)

    # Create nullspace projection matrix
    I = np.eye(W.shape[1])
    nullspace_proj = I - W_pinv @ W

    # Return function that generates solutions for any z
    def solution_generator(z):
        particular_solution = W_pinv @ target
        nullspace_component = nullspace_proj @ z
        return particular_solution + nullspace_component

    return solution_generator

# Create sample data
n = 1024  # Using small dimension for example
W = model.pooler.dense.weight.cpu().detach().clone().numpy()
b = model.pooler.dense.bias.cpu().detach().clone().numpy()
x_original = outputs.hidden_states[24][0][0].cpu().clone().detach().numpy()
y = outputs.pooler_output[0].cpu().clone().detach().numpy()

# Find solution generator
solution_gen = find_all_possible_inputs(y, W, b)

# Generate some solutions
z = np.random.randn(n)
x_solution = solution_gen(z)

# Verify solution
y_reconstructed = np.tanh(W @ x_solution + b)
print("Original y:", y)
print("Reconstructed y:", y_reconstructed)
print("Difference:", np.abs(y - y_reconstructed).max())

Original y: [-0.94499    -0.8097365  -0.83618635 ...  0.48591825  0.9817478
 -0.8716495 ]
Reconstructed y: [-0.94499275 -0.80972935 -0.8361861  ...  0.48590383  0.9817479
 -0.871657  ]
Difference: 0.00014365059180654338


In [44]:
import numpy as np
from scipy.optimize import minimize

def optimize_x(y, W, b):
    """
    Optimize the input x to minimize the difference between reconstructed y and original y.

    Parameters:
        y: Target output vector (all elements between -1 and 1)
        W: Weight matrix
        b: Bias vector

    Returns:
        Optimized x
    """
    # Check if y is valid
    if not np.all(np.abs(y) < 1):
        raise ValueError("All elements of y must be between -1 and 1")

    # Step 1: Apply arctanh to y
    arctanh_y = np.arctanh(y)

    # Step 2: Subtract bias
    target = arctanh_y - b

    # Step 3: Calculate pseudoinverse of W
    W_pinv = np.linalg.pinv(W)
    I = np.eye(W.shape[1])
    nullspace_proj = I - W_pinv @ W

    # Step 4: Initialize x with the particular solution
    x_particular = W_pinv @ target

    # Define the loss function
    def loss(z):
        x_candidate = x_particular + nullspace_proj @ z
        y_reconstructed = np.tanh(W @ x_candidate + b)
        return np.linalg.norm(y - y_reconstructed)

    # Optimize z to minimize the loss
    z0 = np.zeros(W.shape[1])
    result = minimize(loss, z0, method="BFGS")

    # Get the optimized x
    z_optimized = result.x
    x_optimized = x_particular + nullspace_proj @ z_optimized

    return x_optimized, z_optimized

# Example usage with your model data
W = model.pooler.dense.weight.cpu().detach().clone().numpy()
b = model.pooler.dense.bias.cpu().detach().clone().numpy()
y = outputs.pooler_output[0].cpu().clone().detach().numpy()

x_optimized, z_optimal = optimize_x(y, W, b)

# Verify solution
y_reconstructed = np.tanh(W @ x_optimized + b)
print("Original y:", y)
print("Reconstructed y:", y_reconstructed)
print("Difference:", np.abs(y - y_reconstructed).max())
print("Minimized |x|:", np.linalg.norm(x_optimized))

Original y: [-0.94499    -0.8097365  -0.83618635 ...  0.48591825  0.9817478
 -0.8716495 ]
Reconstructed y: [-0.94498777 -0.80972364 -0.83618838 ...  0.48594797  0.98174797
 -0.87165061]
Difference: 7.062228422757832e-05
Minimized |x|: 16.75909870214801


In [30]:
print(model.pooler.dense.weight.cpu().detach().clone().numpy())

[[-0.0390625   0.04443359 -0.0249176  ...  0.00100899  0.02441406
   0.00793457]
 [-0.02255249  0.03729248 -0.03503418 ... -0.01218414  0.04241943
   0.00948334]
 [-0.03277588 -0.01222229 -0.0519104  ... -0.07611084  0.00650787
  -0.01287842]
 ...
 [-0.00329208  0.01369476  0.0128479  ...  0.05535889 -0.06280518
  -0.01118469]
 [-0.03326416  0.01647949 -0.02340698 ... -0.03839111 -0.03436279
  -0.06286621]
 [ 0.0136795   0.00485611 -0.01486206 ...  0.01111603  0.00914001
   0.02174377]]
